### QED-C Application-Oriented Benchmarks - Hamiltonian Simulation with HamLib - Data for Paper

The notebook contains specific examples for the HamLib-based Hamiltonian Simulation benchmark program.
Configure and run the cells below with the desired application and execution settings.

The notebook is designed to loop over many options we would like for generating paper data.
We loop over several a list of different configuration options and save data for a range of qubit sizes.
Note that the settings are configured for execution on a desktop system with GPU card for faster calculations.


In [1]:
%reload_ext autoreload
%autoreload 2

### Program Configuration 

In [ ]:
# Select the API to use and load the benchmark program
api = "qiskit"
from hamlib import hamlib_simulation_benchmark

# Select the benchmark parameters
min_qubits = 4
max_qubits = 8
skip_qubits=1
max_circuits=1
num_shots=10000

# Select the backend 
#backend_id="qasm_simulator"
backend_id="statevector_simulator"
provider_backend=None
exec_options={}

#############

# place resulting data and images into a folder with "paper" suffix
from _common import metrics
metrics.data_suffix = "_paper"
hamlib_simulation_benchmark.data_suffix = "_paper"


### List of Available Hamiltonians and Sizes
For the plotting illustrate below, select one of the available Hamiltonian name, params tuples.
In the cells below, we loop over all these Hamiltonians for testing purposes.

In [3]:
# list of observable computation methods to sweep
group_method_list = [ None, "simple", "N", "estimator" ]

# List of HamLib filename and parameter tuples, with max_qubits limit array
# Note that the 4th entry in the array is used in the cell below which executes a noisy simulation
hamiltonians = [
    ('condensedmatter/tfim/tfim', { "1D-grid": "pbc", "h": 2 }, [24,28,28,28, 16] ),
    ('condensedmatter/heisenberg/heis', { "1D-grid": "pbc", "h": 2 }, [24,28,28,28, 16] ),
    ('condensedmatter/fermihubbard/FH_D-1', { "1D-grid": "pbc", "enc": "bk", "U":12 }, [20,24,24,24, 14] ),
    ('condensedmatter/bosehubbard/BH_D-1_d-4', { "1D-grid": "nonpbc", "enc": "gray", "U":10 }, [16,24,24,24, 12] ),
    ('binaryoptimization/max3sat/random/random_max3sat-hams', { "ratio": "2", "rinst": "02" }, [18,28,28,28, 14] ),
    ('chemistry/electronic/standard/H2', { "ham_BK": "" }, [8,12,12,12, 8] )
    ]

# Select indices of the Hamiltonian(s) to test
ham_indexes = range(len(hamiltonians))       # select all
#ham_indexes = [2,3]       # select subset, for example 2,3; for quick test, do just one of them

# defaults for Trotter evolution
K = 1
t = 0.1


### Perform Hamiltonian Simulations and Measure Expectation Accuracy (Statevector Tests)
Here, we execute the Hamiltonian simulation over an array of different Hamiltonians.
We exercise each of several expectation methods and assess the results.


In [4]:
# Loop over Hamiltonians for testing expectation value computation

for i in ham_indexes:
    hamiltonian_name, hamiltonian_params, max_qubits_array = hamiltonians[ i ]
    
    print(f"\n*******************************************************************************************")

    print(f"\nHamiltonian Simulation Test for {hamiltonian_name} {hamiltonian_params}\n")

    ###################

    for i, group_method in enumerate(group_method_list):

        print("\n============================================================\n")
        print(f"Testing expectation grouping method: {group_method}") 
        
        # limit qubit width based on table, or constant value if no table entry
        max_qubits = max_qubits_array[i] if max_qubits_array else 12

        """
        # for testing notebook logic at smaller sizes
        if max_qubits > 8: max_qubits = max_qubits - 4 
        if max_qubits > 14: max_qubits = 14
        """
        
        # do more qubits if the "simple" test, as this one generates data for paper
        hamlib_simulation_benchmark.max_qubits_exact = 18 if group_method == "simple" else 16      
        
        # execute an instance of the benchmark for given qubit range
        hamlib_simulation_benchmark.run(min_qubits=min_qubits, max_qubits=max_qubits, skip_qubits=skip_qubits,
                    max_circuits=max_circuits, num_shots=num_shots,
                    method=4,
                    draw_circuits = False,
                    plot_results = False,
                                        
                    hamiltonian = hamiltonian_name,
                    hamiltonian_params = hamiltonian_params,
                    K = K, t = t,

                    do_observables = True,
                    group_method = group_method,
                                         
                    backend_id=backend_id,
                    provider_backend=provider_backend,
                    exec_options=exec_options,
                    api=api)

print(f"\n*******************************************************************************************\n")



*******************************************************************************************

Hamiltonian Simulation Test for condensedmatter/tfim/tfim {'1D-grid': 'pbc', 'h': 2}



Testing expectation grouping method: None
Hamiltonian Simulation Benchmark Program - Qiskit
... execution starting at Feb 14, 2025 04:19:44 UTC
************
Executing [1] circuits with num_qubits = 4
... dataset_name = graph-1D-grid-pbc-qubitnodes_Lx-4_h-2
... number of terms in Hamiltonian = 8
... execute_cicuits(statevector_simulator, 8, 10000)
... number of circuits executed = 8
... quantum execution time = 0.048
... exact computation time = 0.0 sec

    Exact expectation value, computed classically: -4.0
    Estimated expectation value, from quantum algorithm: -3.5098
    ==> Simulation Quality: 0.877

************
Executing [1] circuits with num_qubits = 5
... dataset_name = graph-1D-grid-pbc-qubitnodes_Lx-5_h-2
... number of terms in Hamiltonian = 10
... execute_cicuits(statevector_simulator, 10, 1000

### Perform Hamiltonian Simulations and Measure Expectation Accuracy (Noisy Simulator Tests)
Here, we execute the Hamiltonian simulation over an array of different Hamiltonians.
We only the "simple" grouping method and assess the results.


In [5]:
# Loop over Hamiltonians for testing expectation value computation

# Here we collect data on the noisy simulator
backend_id = "qasm_simulator"
hamlib_simulation_benchmark.data_suffix = "_noisy_paper" 

for i in ham_indexes:
    hamiltonian_name, hamiltonian_params, max_qubits_array = hamiltonians[ i ]
    
    print(f"\n*******************************************************************************************")

    print(f"\nHamiltonian Simulation Test for {hamiltonian_name} {hamiltonian_params}\n")

    ###################

    # on QASM simulator we only do simple grouping method (for now)
    group_method_list_sim = [ "simple" ]

    for i, group_method in enumerate(group_method_list_sim):

        print("\n============================================================\n")
        print(f"Testing expectation grouping method: {group_method}") 
        
        # limit qubit width based on table, or constant value if no table entry
        
        # ************ the 5th entry in the list has the limits for simulator
        max_qubits = max_qubits_array[ i + 4 ] if max_qubits_array else 12

        """
        # for testing notebook logic at smaller sizes
        if max_qubits > 8: max_qubits = max_qubits - 4 
        if max_qubits > 14: max_qubits = 14
        """
        
        # do more qubits if the "simple" test, as this one generates data for paper
        hamlib_simulation_benchmark.max_qubits_exact = 18 if group_method == "simple" else 16      
        
        # execute an instance of the benchmark for given qubit range
        hamlib_simulation_benchmark.run(min_qubits=min_qubits, max_qubits=max_qubits, skip_qubits=skip_qubits,
                    max_circuits=max_circuits, num_shots=num_shots,
                    method=4,
                    draw_circuits = False,
                    plot_results = False,
                                        
                    hamiltonian = hamiltonian_name,
                    hamiltonian_params = hamiltonian_params,
                    K = K, t = t,

                    do_observables = True,
                    group_method = group_method,
                                         
                    backend_id=backend_id,
                    provider_backend=provider_backend,
                    exec_options=exec_options,
                    api=api)

print(f"\n*******************************************************************************************\n")



*******************************************************************************************

Hamiltonian Simulation Test for condensedmatter/tfim/tfim {'1D-grid': 'pbc', 'h': 2}



Testing expectation grouping method: simple
Hamiltonian Simulation Benchmark Program - Qiskit
... execution starting at Feb 14, 2025 04:33:03 UTC
************
Executing [1] circuits with num_qubits = 4
... dataset_name = graph-1D-grid-pbc-qubitnodes_Lx-4_h-2
... number of terms in Hamiltonian = 8
... execute_cicuits(qasm_simulator, 2, 10000)
... number of circuits executed = 2
... quantum execution time = 0.016
... exact computation time = 0.0 sec

    Exact expectation value, computed classically: -4.0
    Estimated expectation value, from quantum algorithm: -3.0391
    ==> Simulation Quality: 0.76

************
Executing [1] circuits with num_qubits = 5
... dataset_name = graph-1D-grid-pbc-qubitnodes_Lx-5_h-2
... number of terms in Hamiltonian = 10
... execute_cicuits(qasm_simulator, 2, 10000)
... number 

In [6]:
import qiskit
print(qiskit.__version__)

1.2.1
